In [3]:
import json
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # this is used for the plot the graph 
from numpy import median
import matplotlib.pyplot as plt
import warnings

In [10]:
#loads the movie dataset with release date
def load_tmdb_movies(path):
    file = pd.read_csv(path)
    file['release_date'] = pd.to_datetime(file['release_date']).apply(lambda x: x.date()) #convert to date time for us
    json_columns = ['genres', 'keywords', 'production_countries', 'production_companies', 'spoken_languages'] #these are json cols
    for column in json_columns:
        file[column] = file[column].apply(json.loads)
    return file

print(load_tmdb_movies("./tmdb-movie-metadata/tmdb_5000_movies.csv"))

         budget                                             genres  \
0     237000000  [{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...   
1     300000000  [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...   
2     245000000  [{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...   
3     250000000  [{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...   
4     260000000  [{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...   
...         ...                                                ...   
4798     220000  [{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...   
4799       9000  [{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...   
4800          0  [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...   
4801          0                                                 []   
4802          0                [{'id': 99, 'name': 'Documentary'}]   

                                               homepage      id  \
0                           http://www.avatarmovie.com/   19995   
1          http://disney.

In [8]:
def load_tmdb_credits(path):
    file = pd.read_csv(path)
    json_columns = ['cast', 'crew']
    for column in json_columns:
        file[column] = file[column].apply(json.loads)
    return file

print(load_tmdb_credits("./tmdb-movie-metadata/tmdb_5000_credits.csv"))

      movie_id                                     title  \
0        19995                                    Avatar   
1          285  Pirates of the Caribbean: At World's End   
2       206647                                   Spectre   
3        49026                     The Dark Knight Rises   
4        49529                               John Carter   
...        ...                                       ...   
4798      9367                               El Mariachi   
4799     72766                                 Newlyweds   
4800    231617                 Signed, Sealed, Delivered   
4801    126186                          Shanghai Calling   
4802     25975                         My Date with Drew   

                                                   cast  \
0     [{'cast_id': 242, 'character': 'Jake Sully', '...   
1     [{'cast_id': 4, 'character': 'Captain Jack Spa...   
2     [{'cast_id': 1, 'character': 'James Bond', 'cr...   
3     [{'cast_id': 2, 'character': 'Bruce W